## Pruning
<div class='alert alert-success'>
Ce petit notebook permet d'évaluer AbstractWeight sur un petit réseau. 
Pour cela, on créé AbstractNN, un petit réseau composé de 3 couches de convolution suivies de ReLU, puis de deux couches FC suivies également de ReLU. 
Pour l'entrainement uniquement, un softmax est appliqué en sortie de réseau
</div>

Dans la classe AbstractNN, la méthode forward de nn.Linear est surchargée de façon standard.
C'est à dire que si l'on instancie model=AbstractNN(), model(x) applique forward à x comme dans n'importe quel réseau. 

AbstractNN possède une méthode abstract_forward() qui fait appelle à AbstractLinear et AbstractReLU, AbstractMaxpool et AbstractWeights.

Le réseau est préalablement entraîné. On va tenter d'analyser le comportement du réseau autour des poids que l'on souhaite retirer. Pour cela, on recherche premièrement les candidats à l'élagage (méthode heuristique, poids les plus faibles en valeur absolu). On en selectionne une quantité n, puis on analyse le comportement du réseau autour de ces poids: 

$$\forall p\in [1,m], y_p = \sum_{k=1}^n w_{pk} x_k +\alpha_{pk}x_k  \delta_{pk} $$

On utilise pour cela la classe AbstractWeight, qui convertit une couche FC dans un réseau en domaine abstrait. Cette classe prend en argument l'index des poids à bruiter, les valeurs du bruits. 
Dans le cadre de l'élagage, on souhaite que l'expression affine inclue la valeur pour laquelle le poids est nul. On choisit donc un alpha de bruit valant le poids en valeur absolu plus un coeficient de marge (10%). 
On est donc certain que l'expression affine en sortie du réseau inclue l'image de la matrice élaguée. 

Enfin on observe la relevance sur la matrice de poids. Cette relevance est normalisée par la valeur de alpha. 
Les indices ayant une forte relevance ont donc contribué à la dispersion du domaine, ils affectent en premier lieu la dominance.

On étudiera par la suite une méthode pour retirer de la liste des candidats à l'élagage les poids contribuants le plus à la réduction de la dominance. 



In [ ]:


%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
sys.path.append('../util')
import torch
import torch.nn as nn

from abstractModule import AbstractLinear as AL
from abstractModule import AbstractReLU as AR
from abstractModule import AbstractMaxpool2D as AM
from abstractWeight import AbstractWeight as AW

from custom_train import CustomTrainer as T
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import random_split
from torch import optim
import os


device = torch.device('cpu')

Ici on importe le dataset FashionMNIST normalisation et random split pour l'entrainement

In [ ]:

path ='dataset'

os.makedirs(path, exist_ok=True)

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean =[0.5], std =[0.2]),
        #transforms.Resize((56,56))
    ])



dataset_train = datasets.FashionMNIST(root = path,transform = transform, download = True, train = True)
dataset_test =datasets.FashionMNIST( root =path,transform=transform ,download = True, train = False)
val =0.2
len_data_train = len(dataset_train)
train_size =int((1-val)*len_data_train)

val_size = int(val*len_data_train)

train_dataset, val_dataset = random_split(dataset_train, [train_size,val_size])


Ici on génère notre classe AbstractNN et sa fameuse méthode abstrac_forward


In [ ]:


class AbstractNN(nn.Module):
    
    def __init__(self,num_depth=1,device=torch.device("cpu")):

        super(AbstractNN,self).__init__()
       
      
        self.num_depth = num_depth
        self.device = device
        self.conv1=nn.Conv2d(self.num_depth,16,3,device=self.device)
        self.conv2=nn.Conv2d(16,32,3,device=self.device)
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
  

       

        self.fc1=nn.Sequential(nn.Flatten(),nn.Linear(4608,64,device=self.device))
        self.fc2=nn.Sequential(nn.Flatten(),nn.Linear(64,10,device=self.device))
        self.softMax =nn.Softmax()
    def forward(self,x):
        x=self.conv1(x)
        x=torch.relu(x)
        x=self.conv2(x)
        x=torch.relu(x)
    
        x=self.maxpool(x)
   
    
        x=self.fc1(x)
        x=torch.relu(x)
        x=self.fc2(x)
        x=torch.relu(x)
        x= self.softMax(x)
        return x
    
    def abstract_forward(self,x,add_symbol=False,device=torch.device("cpu"),index=None,alpha=None):
        
        
        self.device=device
        #Dans cette première partie, du réseau, on reste dans le domaine concrêt.
        x=self.conv1(x)
        x=torch.relu(x)
        x=self.conv2(x)
        x=torch.relu(x)
        x=self.maxpool(x)
    
        x_true = x
        x_true = x_true[0].unsqueeze(0)

        
        print(f"x.shape = {x.shape}")
        x,x_min,x_max,x_true = AW.generate_ztp_from_layer_and_indexes(self.fc1,index =index.indices,alpha = alpha,input = x, device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
      
        x,x_min,x_max,x_true = AL.abstract_linear(self.fc2,x,x_true,device=self.device)
        x,x_min,x_max,x_true = AR.abstract_relu(x,x_min,x_max,x_true,add_symbol=add_symbol,device =self.device)
       
        
        return x,x_min,x_max,x_true
        

Un petit script pour l'entrainement, disons 10 EPOCH

In [ ]:
"""
model =AbstractNN(num_depth=1,device=device)

num_epochs =10
learning_rate=0.01
batch_size = 128

optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.99))
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.98)
criterion = nn.CrossEntropyLoss()
TD= T(model=model,
            device=device
                            )
model = TD.train_model(train_dataset,val_dataset,
                        criterion=criterion,  
                        batch_size= batch_size,
                        optimizer=optimizer,
                        scheduler=scheduler,
                        num_epochs=num_epochs,
                        learning_rate=learning_rate,
                        resname='SimpleCNN',verbose=False)
"""


In [ ]:
model=AbstractNN(num_depth=1,device=device   )
model.load_state_dict(torch.load('dataset/test.pth'))

model.to(device)

On récupère la matrice de poids de fc1


In [ ]:
for param in model.named_parameters():
   if param[0] == 'fc1.1.weight':
        print(torch.sum(param[1].data))        
        index = torch.topk(torch.abs(param[1].data).flatten(),k=100_000,largest=False)
        mat= param[1].data
        break
print(index.indices)
alpha= 1.1*torch.abs(index.values)
print(alpha)
print(f'mat.shape = {mat.shape}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

img, label = dataset_train[230]
plt.imshow(1-img[0],cmap='gray')
print(f"Label:{label}")
img.shape

In [ ]:
model.eval()
model.to(device)
img=img.to(device)
print(img.unsqueeze(0).shape)
with torch.no_grad():

 
   result,x_min,x_max,x_true=model.abstract_forward(img.unsqueeze(0),add_symbol=True,device=device,index=index,alpha=alpha)
print(f"result.shape = {result.shape}")
print(f"y_min       =  {x_min}")
print(f"y_max       =  {x_max}")
print(f"center Ztp  =  {result[0]}")
print(f"y_true      =  {x_true[:]}")
print(f"y_max-x_min =  {x_max-x_min}")
print(f"Trash symbol=  {result[-1]}")

In [ ]:

y_min       =  np.array(x_min)
y_max       =  np.array(x_max)
center_Ztp  =  np.expand_dims(np.array(result[0]),axis =1)
y_true      =  np.expand_dims(np.array(x_true[:])[0],axis =1)
y_max_minus_y_min =  np.array(x_max-x_min)
Trash_symbol=  np.array(result[-1])


import matplotlib.pyplot as plt
import numpy as np

#plt.style.use('_mpl-gallery')

# make data:

x = np.arange(len(y_min))
D =np.stack((y_min,y_max),axis=1)

print(D.shape)
print(center_Ztp.shape)
print(y_true.shape)
# plot:

fig,ax = plt.subplots(1, 1, figsize=(8,4), tight_layout=True)
ax.eventplot(D, orientation="vertical", linewidth=1,color='blue',linelengths=0.3)
ax.eventplot(y_true, orientation="vertical", linewidth=0.50,color='green',linelengths=0.4)
ax.eventplot(center_Ztp, orientation="vertical", linewidth=1,color='red',linelengths=0.5)

ax.set(xlim=(0, 10),xticks=x,xticklabels=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag,","Ankle boot"],
       ylim=(-20, 40))
plt.ylabel("Value of the abstract domain")
plt.title("Dominance interval for the 10 classes of Fashion MNIST .\n Abstract domain based on 100_000 lower weights of the first layer of the first fully connected layer of the model")
plt.legend()
plt.show()

Voici le premier affichage de relevance sur l'image d'une couche calculé par AbstractTorch

In [ ]:
print(f"mat.shape = {mat.shape}")
map_relevance = torch.zeros_like(mat).flatten()
map_original = torch.zeros_like(mat).flatten() 

value=torch.argmax(result[0])
print(f"result[1:100001,value]= {result[1:100001,value].shape}")
map_relevance[index.indices]= result[1:100001,value]
map_original[index.indices]= mat.flatten()[index.indices]
map_relevance = map_relevance.reshape(mat.shape)
map_original = map_original.reshape(mat.shape)
print(f"map_2.shape = {map_relevance.shape}")

map_normalized_relevance = torch.abs(map_relevance)/torch.abs(map_original)
map_normalized_relevance = torch.where(torch.isnan(map_normalized_relevance), torch.zeros_like(map_normalized_relevance), map_normalized_relevance)



from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.figure(figsize=(12, 6)) 

im = plt.imshow(torch.abs(map_normalized_relevance.cpu())[:, 200:600].numpy(), cmap='viridis')

plt.title('Normalised_Relevance')


divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)  


plt.colorbar(im, cax=cax)

plt.show()